In [1]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [2]:
dict_table = pickle.load(open('data/dict_table','rb'))
df_historical_data = pd.read_csv('data/clean_fifa_worldcup_matches.csv')
df_fixture = pd.read_csv('data/clean_fifa_worldcup_fixture.csv')

# Calculate Team Strength

In [3]:
df_home = df_historical_data[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = df_historical_data[['AwayTeam', 'HomeGoals', 'AwayGoals']]

df_home = df_home.rename(columns={'HomeTeam':'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals': 'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team', 'HomeGoals': 'GoalsConceded', 'AwayGoals': 'GoalsScored'})

df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby(['Team']).mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.691358,1.148148
Australia,0.812500,1.937500
Austria,1.482759,1.620690
...,...,...
Uruguay,1.553571,1.321429
Wales,0.800000,0.800000
West Germany,2.112903,1.241935


# Function predict_points

In [4]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

## Testing function

In [5]:
print(predict_points('England', 'United States'))
print(predict_points('Argentina', 'Mexico'))
print(predict_points('Qatar (H)', 'Ecuador')) # Qatar vs Team X -> 0 points to both

(2.2356147635326007, 0.5922397535606193)
(2.3129151525530505, 0.5378377125059863)
(0, 0)


# Predicting World Cup

## Group stage

In [6]:
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [7]:
dict_table['Group A']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Netherlands,1,1,0,0,2,0,+2,3[a]
1,2,Ecuador,1,1,0,0,2,0,+2,3[a]
2,3,Senegal,1,0,0,1,0,2,−2,0[b]
3,4,Qatar (H),1,0,0,1,0,2,−2,0[b]


In [8]:
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        print("current group:>>>>", group)
        print('home and away:%s,%s'%(home,away))
        print("before frame:",dict_table[group])
        try:  
            dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
            dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away
        except:
            a=dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts']
            for x in a.str.split():
                aa= x[0].split('[')[0]
                print(aa)
            aa = float(aa) + points_home
            dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] = str(aa)
            b = dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts']
            for y in b.str.split():
                bb = y[0].split('[')[0]
            bb = float(bb) +points_away
            dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] = str(bb)
        print("after frame:",dict_table[group])
            
            #print('a',a)
            #dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] = a+ points_home
            #b = float(dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'].str.split("[")[0][0])
            #print('b',b)
            #dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] = b+points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

current group:>>>> Group A
home and away:Senegal,Netherlands
before frame:    Pos         Team  Pld  W  D  L  GF  GA  GD   Pts
0    1  Netherlands    1  1  0  0   2   0  +2  3[a]
1    2      Ecuador    1  1  0  0   2   0  +2  3[a]
2    3      Senegal    1  0  0  1   0   2  −2  0[b]
3    4    Qatar (H)    1  0  0  1   0   2  −2  0[b]
0
after frame:    Pos         Team  Pld  W  D  L  GF  GA  GD                 Pts
0    1  Netherlands    1  1  0  0   2   0  +2   4.897481362825154
1    2      Ecuador    1  1  0  0   2   0  +2                3[a]
2    3      Senegal    1  0  0  1   0   2  −2  0.8966936039774208
3    4    Qatar (H)    1  0  0  1   0   2  −2                0[b]
current group:>>>> Group A
home and away:Netherlands,Ecuador
before frame:    Pos         Team  Pld  W  D  L  GF  GA  GD                 Pts
0    1  Netherlands    1  1  0  0   2   0  +2   4.897481362825154
1    2      Ecuador    1  1  0  0   2   0  +2                3[a]
2    3      Senegal    1  0  0  1   0   2  −2  

current group:>>>> Group C
home and away:Mexico,Poland
before frame:    Pos          Team  Pld  W  D  L  GF  GA  GD                 Pts
0    1  Saudi Arabia    1  1  0  0   2   1  +1  3.1482917658633607
1    2        Poland    1  0  1  0   0   0   0                1[a]
2    3        Mexico    1  0  1  0   0   0   0                1[a]
3    4     Argentina    1  0  0  1   1   2  −1    2.78159351546888
1
after frame:    Pos          Team  Pld  W  D  L  GF  GA  GD                 Pts
0    1  Saudi Arabia    1  1  0  0   2   1  +1  3.1482917658633607
1    2        Poland    1  0  1  0   0   0   0   2.944730740412008
2    3        Mexico    1  0  1  0   0   0   0  1.8604264483835886
3    4     Argentina    1  0  0  1   1   2  −1    2.78159351546888
current group:>>>> Group C
home and away:Poland,Saudi Arabia
before frame:    Pos          Team  Pld  W  D  L  GF  GA  GD                 Pts
0    1  Saudi Arabia    1  1  0  0   2   1  +1  3.1482917658633607
1    2        Poland    1  0  1  0   

after frame:    Pos        Team  Pld  W  D  L  GF  GA  GD       Pts
0    1       Spain    1  1  0  0   7   0  +7  3.000000
1    2       Japan    1  1  0  0   2   1  +1  3.456492
2    3     Germany    1  0  0  1   1   2  −1  2.398799
3    4  Costa Rica    1  0  0  1   0   7  −7  0.000000
current group:>>>> Group E
home and away:Spain,Costa Rica
before frame:    Pos        Team  Pld  W  D  L  GF  GA  GD       Pts
0    1       Spain    1  1  0  0   7   0  +7  3.000000
1    2       Japan    1  1  0  0   2   1  +1  3.456492
2    3     Germany    1  0  0  1   1   2  −1  2.398799
3    4  Costa Rica    1  0  0  1   0   7  −7  0.000000
after frame:    Pos        Team  Pld  W  D  L  GF  GA  GD       Pts
0    1       Spain    1  1  0  0   7   0  +7  5.118647
1    2       Japan    1  1  0  0   2   1  +1  3.456492
2    3     Germany    1  0  0  1   1   2  −1  2.398799
3    4  Costa Rica    1  0  0  1   0   7  −7  0.699844
current group:>>>> Group E
home and away:Japan,Costa Rica
before frame:    Po

current group:>>>> Group G
home and away:Cameroon,Serbia
before frame:    Pos         Team  Pld  W  D  L  GF  GA  GD       Pts
0    1       Brazil    0  0  0  0   0   0   0  2.466296
1    2       Serbia    0  0  0  0   0   0   0  0.383283
2    3  Switzerland    0  0  0  0   0   0   0  2.067867
3    4     Cameroon    0  0  0  0   0   0   0  0.751699
after frame:    Pos         Team  Pld  W  D  L  GF  GA  GD       Pts
0    1       Brazil    0  0  0  0   0   0   0  2.466296
1    2       Serbia    0  0  0  0   0   0   0  1.991065
2    3  Switzerland    0  0  0  0   0   0   0  2.067867
3    4     Cameroon    0  0  0  0   0   0   0  1.854774
current group:>>>> Group G
home and away:Brazil,Switzerland
before frame:    Pos         Team  Pld  W  D  L  GF  GA  GD       Pts
0    1       Brazil    0  0  0  0   0   0   0  2.466296
1    2       Serbia    0  0  0  0   0   0   0  1.991065
2    3  Switzerland    0  0  0  0   0   0   0  2.067867
3    4     Cameroon    0  0  0  0   0   0   0  1.854774
af

In [9]:
dict_table['Group B']

,Team,Pts
0,England,9.217753127575735
1,Wales,5.878763834176662
2,United States,4.349248480265582
3,Iran,2.0922887730275064


## Knock out

In [10]:
df_fixture_knockout

,home,score,away,year
48,Winners Group A,Match 49,Runners-up Group B,2022
49,Winners Group C,Match 50,Runners-up Group D,2022
50,Winners Group D,Match 52,Runners-up Group C,2022
51,Winners Group B,Match 51,Runners-up Group A,2022
52,Winners Group E,Match 53,Runners-up Group F,2022
53,Winners Group G,Match 54,Runners-up Group H,2022
54,Winners Group F,Match 55,Runners-up Group E,2022
55,Winners Group H,Match 56,Runners-up Group G,2022


In [11]:
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    df_fixture_knockout.replace({f'Winners {group}':group_winner,
                                 f'Runners-up {group}':runners_up}, inplace=True)

df_fixture_knockout['winner'] = '?'
df_fixture_knockout

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,?
49,Argentina,Match 50,Denmark,2022,?
50,France,Match 52,Poland,2022,?
51,England,Match 51,Ecuador,2022,?
52,Spain,Match 53,Croatia,2022,?
53,Brazil,Match 54,Uruguay,2022,?
54,Belgium,Match 55,Germany,2022,?
55,Portugal,Match 56,Switzerland,2022,?


In [12]:
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated

In [13]:
get_winner(df_fixture_knockout)

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Ecuador,2022,England
52,Spain,Match 53,Croatia,2022,Spain
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Belgium,Match 55,Germany,2022,Germany
55,Portugal,Match 56,Switzerland,2022,Portugal


## Quarter Final

In [14]:
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winners {match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

In [15]:
update_table(df_fixture_knockout, df_fixture_quarter)

,home,score,away,year,winner
56,Spain,Match 58,Brazil,2022,?
57,Netherlands,Match 57,Argentina,2022,?
58,Germany,Match 60,Portugal,2022,?
59,England,Match 59,France,2022,?


In [16]:
get_winner(df_fixture_quarter)

,home,score,away,year,winner
56,Spain,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Germany,Match 60,Portugal,2022,Germany
59,England,Match 59,France,2022,France


## Semifinal

In [17]:
update_table(df_fixture_quarter, df_fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,?
61,France,Match 62,Germany,2022,?


In [18]:
get_winner(df_fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,Brazil
61,France,Match 62,Germany,2022,Germany


## Final

In [19]:
update_table(df_fixture_semi, df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Brazil,Match 64,Germany,2022,?


In [20]:
get_winner(df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Brazil,Match 64,Germany,2022,Brazil
